In [ ]:
import argparse

import sys

import tensorflow as tf

sys.path.insert(0, r"../utilities")
sys.path.insert(0, r"../preprocessing")
sys.path.insert(0, r"../trainers")

from preprocessor import *
from utils import loss_accuracy_plot

from Seq2SeqTrainer import Seq2SeqTrainer

encoder_tokenizer_name = "en_tokenizer"
decoder_tokenizer_name = "fr_tokenizer"

tf.get_logger().setLevel('WARNING')

In [ ]:
data_dir = "../data"
en_lines, fr_lines = read_data_files(data_dir, ("small_vocab_en", "small_vocab_fr"))

train_data, test_data, prediction_data, tokenizers = preprocess_data(en_lines, fr_lines, [encoder_tokenizer_name, decoder_tokenizer_name])

In [ ]:
LSTM_SIZE = 512
EMBEDDING_SIZE = 250
BATCH_SIZE= 64
EPOCHS = 20

In [ ]:
trainer = Seq2SeqTrainer(batch_size=BATCH_SIZE, 
                         lstm_size=LSTM_SIZE, 
                         embedding_size=EMBEDDING_SIZE, 
                         tokenizers=tokenizers, 
                         predict_every=2)

In [ ]:
losses, accuracy = trainer.train(train_data=train_data,
                                test_data=test_data,
                                prediction_data=prediction_data,
                                epochs=EPOCHS)

In [ ]:
loss_accuracy_plot(losses, accuracy, "Seq2Seq")